In [ ]:
from google.colab import files
import os

: 

In [ ]:
files.upload()

: 

In [ ]:
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

: 

In [ ]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

: 

In [ ]:
!unzip \*.zip

: 

In [ ]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('D:/Games/cell_images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


: 

In [ ]:
infected = '/content/cell_images/cell_images/Parasitized'
uninfected = '/content/cell_images/cell_images/Uninfected'

: 

In [ ]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt

: 

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D,MaxPooling2D
# from tensorflow.keras.applications.vgg19 import VGG19

: 

In [ ]:
# data=pd

: 

In [ ]:
infected_images = []
for file in os.listdir(infected):
    if file == 'Thumbs.db':
        pass
    else:
        img = Image.open(os.path.join(infected, file))
        img = img.resize((36,36))
        img = np.asarray(img)
        infected_images.append(img)

: 

In [ ]:
uninfected_images = []
for file in os.listdir(uninfected):
    if file == 'Thumbs.db':
        pass
    else:
        img = Image.open(os.path.join(uninfected, file))
        img = img.resize((36,36))
        img = np.asarray(img)
        uninfected_images.append(img)

: 

In [ ]:
for i in range(5):
    plt.imshow(infected_images[i])
    plt.show()

: 

In [ ]:
for i in range(5):
    plt.imshow(uninfected_images[i])
    plt.show()

: 

In [ ]:
images = np.asarray(infected_images + uninfected_images)
images.shape

: 

In [ ]:
labels = np.asarray([1 for _ in range(len(infected_images))] + [0 for _ in range(len(uninfected_images))])
labels.shape

: 

In [ ]:
from sklearn.utils import shuffle

: 

In [ ]:
images, labels = shuffle(images, labels)

: 

In [ ]:
for i in range(10):
    print(labels[i])
    plt.imshow(images[i])
    plt.show()

: 

In [ ]:
# np.save('images.npy', images)
# np.save('labels.npy', labels)

: 

In [ ]:
from tensorflow.keras.utils import to_categorical

: 

In [ ]:
labels = to_categorical(labels, num_classes = 2)
labels[:10]

: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

: 

In [ ]:
print("Train size:", X_train.shape, y_train.shape)
print("Test size:", X_test.shape, y_test.shape)

: 

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', input_shape = X_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(512, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(96, activation='relu'))

model.add(Dense(2, activation='softmax'))

: 

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

: 

In [ ]:
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), verbose = 1)

: 

In [ ]:
pred = [np.argmax(i) for i in model.predict(X_test)]
pred[:5]

: 

In [ ]:
tru = [np.argmax(i) for i in y_test]
from sklearn.metrics import confusion_matrix
confusion_matrix(tru, pred)

: 

In [ ]:
model.save('malaria.h5')

: 

: 